In [4]:
import math
import json
import pickle
import random
from gensim import models
from gensim import matutils
import logging
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time
from nltk.tokenize import sent_tokenize
import glob
import argparse
import os


In [2]:
### Drap the review whcih has same business id in business object with 'Restaurants' categories.
business_id = []
business_output = []
with open("yelp_academic_dataset.json", 'r') as f:
    for line in f.readlines():
        business_json = json.loads(line)
        if 'categories' in business_json:
            business_output.append(line)
            if 'Restaurants' in business_json['categories']:
                business_id.append(business_json['business_id'])
print "Business Finished"

Business Finished


In [3]:
review_output = []
with open("yelp_academic_dataset.json", 'r') as f:
    for line in f.readlines():
        review_json = json.loads(line)
        if 'text' in review_json:
            if review_json['business_id'] in business_id:
                review_output.append(line)
with open ( "yelp_academic_dataset_review.json", 'w' ) as f:
    f.write(''.join(review_output))
print "Review Finished"

Review Finished


In [33]:
path2buisness="yelp_academic_dataset_business.json"
path2reviews="yelp_academic_dataset_review.json"

categories = set([])
restaurant_ids = set([])
cat2rid = {}
rest2rate={}
rest2revID = {}
r = 'Restaurants'
with open (path2buisness, 'r') as f:
    for line in f.readlines():
        business_json = json.loads(line)
        bjc = business_json['categories']
        #cities.add(business_json['city'])
        if r in bjc:
            if len(bjc) > 1:
                #print(bjc)
                restaurant_ids.add(business_json['business_id'])
                categories = set(bjc).union(categories) - set([r])
                stars = business_json['stars']
                rest2rate[ business_json['business_id'] ] = stars
                for cat in bjc:
                    if cat == r:
                        continue
                    if cat in cat2rid:
                        cat2rid[cat].append(business_json['business_id'])
                    else:
                        cat2rid[cat] = [business_json['business_id']]

print "saving restaurant ratings"
with open ( 'restaurantIds2ratings.txt', 'w') as f:
    for key in rest2rate:
        f.write( key + " " + str(rest2rate[key]) + "\n")
    #clearing from memory
rest2rate=None
with open('data_cat2rid.pickle', 'wb') as f:
    pickle.dump(cat2rid,f)

with open (path2reviews, 'r') as f:
    for line in f.readlines():
        review_json = json.loads(line)
        if review_json['business_id'] in restaurant_ids:
            if review_json['business_id'] in rest2revID:
                rest2revID[ review_json['business_id'] ].append(review_json['review_id'])
            else:
                rest2revID[ review_json['business_id'] ] = [ review_json['review_id'] ]

with open('data_rest2revID.pickle', 'wb') as f:
    pickle.dump(rest2revID,f)

nz_count = 0
valid_cats = []
for i, cat in enumerate(cat2rid):
    cat_total_reviews = 0
    for rid in cat2rid[cat]:
        #number of reviews for each of restaurants
        if rid in rest2revID:
            cat_total_reviews = cat_total_reviews + len(rest2revID[rid])

    if cat_total_reviews > 30:
        nz_count = nz_count + 1
        valid_cats.append(cat)
        #print( cat, cat_total_reviews)


print "finish process"

saving restaurant ratings
finish process


In [35]:
sample_size = 10 #len(valid_cats) # This specifies how many cuisines you would like to save
                                  # if this process takes too long you can change it to something smaller like 5, 6 ...
    
#print nz_count, ' non-zero number of reviews in categories out of', len(cat2rid), 'categories')
#x = range(nz_count)
###def sample_categories(sample_sizes = 10):
print "sampling categories"
sample_rid2cat={}

cat_sample = random.sample(valid_cats, sample_size)
for cat in cat_sample:
    for rid in cat2rid[cat]:
        if rid in rest2revID:
            if rid not in sample_rid2cat:
                sample_rid2cat[rid] = []
            sample_rid2cat[rid].append(cat)
#remove from memory
rest2revID=None
#    print (len(sample_rid2cat), len(cat2rid), len(valid_cats), len(cat_sample))
    
print "reading from reviews file..."
#ensure categories is a directory
sample_cat2reviews={}
sample_cat2ratings={}
num_reviews = 0
with open (path2reviews, 'r') as f:
    for line in f.readlines():
        review_json = json.loads(line)
        rid = review_json['business_id']
        if rid in sample_rid2cat:
            for rcat in sample_rid2cat [ rid ]:
                num_reviews = num_reviews + 1
                if rcat in sample_cat2reviews:
                    sample_cat2reviews [ rcat ].append(review_json['text'])
                    sample_cat2ratings [ rcat ].append( str(review_json['stars']) )
                else:
                    sample_cat2reviews [ rcat ] = [review_json['text']]
                    sample_cat2ratings [ rcat ] = [ str(review_json['stars']) ]

print "finish reading reviews"

sampling categories
reading from reviews file...
finish reading reviews


In [36]:
print "saving categories"
#save categories
for cat in sample_cat2reviews:
    with open ('categories/' + cat.replace('/', '-').replace(" ", "_") + ".txt" , 'w') as f:
        f.write(u'\n'.join(sample_cat2reviews[cat]).encode('utf-8').strip())

print "finish saving"

saving categories
finish saving


In [40]:
#save sample for restaurant reviews
print "sampling restaurant reviews"
def sample_reviews(rate_sizes = 100000):
    sample_size = min(rate_sizes, num_reviews)
    rev_sample = random.sample(range(num_reviews), sample_size)
    my_sample_v2 = []
    sample_ratings = []
    sorted_rev_sample = sorted(rev_sample)
    count = 0
    max_bound = 0
    for cat in sample_cat2reviews:
        print cat
        new_max_bound = max_bound + len(sample_cat2reviews[cat])
        while count < sample_size and sorted_rev_sample[count] < new_max_bound:
            my_sample_v2.append( sample_cat2reviews[cat][ sorted_rev_sample[count] - max_bound ].replace("\n", " ").strip() )
            sample_ratings.append( sample_cat2ratings[cat][ sorted_rev_sample[count] - max_bound ] )
            count = count + 1
        max_bound = new_max_bound
        #if count in rev_sample:
        #    my_sample.append(rev.replace("\n", " ").strip())
        #count = count + 1

    with open ("review_sample_" + str(rate_sizes) + ".txt", 'w') as f:
        f.write('\n'.join(my_sample_v2).encode('ascii','ignore') )

    with open ("review_ratings_" + str(rate_sizes) + ".txt", 'w') as f:
        f.write('\n'.join(sample_ratings).encode('ascii','ignore') )
        
sample_reviews()

sampling restaurant reviews
Portuguese
Music & DVDs
Bistros
Tapas/Small Plates
Beer, Wine & Spirits
Ethiopian
Japanese
British
Hot Dogs
Creperies
